# Retrieval
文档开篇指出，LLM是很牛了，但还有俩关键的不足之处：
- 上下文有限：不能一次摄取全部的预料
- 静态知识：只能依赖训练语料

Retrieval-Augmented Generation (RAG)：enhancing an LLM’s answers with context-specific information.

以下为 相关步骤：
- Document loaders
- ⭐Text splitters
- Embedding models
- ⭐Vector stores
- ⭐Retrievers

## 1 Document loaders

All document loaders implement the [`BaseLoader`](https://reference.langchain.com/python/langchain_core/document_loaders/#langchain_core.document_loaders.BaseLoader) interface.

### Interface
所有 文档加载器 都有一个共享的 API ：
- .load() – Loads all documents at once.
- .lazy_load() – Streams documents lazily, useful for large datasets.

查看支持的 [`文档加载器`](https://docs.langchain.com/oss/python/integrations/document_loaders#by-category)

## 2 Text splitters
将大的文档分割为小的 chunk，以供独立检索并 适配大模型上下文窗口限制。 主要有以下三类分割策略：
### 2.1. 文本结构分割（Text Structure-based）
基于文本的自然层级结构进行分割。用 [`RecursiveCharacterTextSplitter`](https://docs.langchain.com/oss/python/integrations/splitters/recursive_text_splitter) 实现。        
它按照 递归 的方式处理：        

首先尝试保持较大单位完整（如段落）如果超过块大小，降级到句子级别，继续向下降级到单词级别：      

````py
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # 块大小
        chunk_overlap=200,      # 块之间的重叠（保留上下文）
        add_start_index=True    # 保留原始文档中的起始位置
    )
    chunks = text_splitter.split_documents(docs)

    # 默认的分隔符列表顺序（英文）
    separators = [
        "\n\n",      # 段落分隔符（空白行）- 优先级最高
        "\n",        # 句子分隔符（换行符）
        " ",         # 单词分隔符（空格）
        ""           # 字符分隔符（最后手段）
    ]
````

### 2.2. 长度分割（Length-based）
根据长度限制（字符数或 token 数）进行分割。         
- 字符分割
````py
    from langchain_text_splitters import CharacterTextSplitter

    text_splitter = CharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0
    )
    chunks = text_splitter.split_text(document)
````
- Token 分割
````py
    from langchain_text_splitters import CharacterTextSplitter

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        encoding_name="cl100k_base",  # OpenAI 编码
        chunk_size=100,               # Token 数量
        chunk_overlap=0
    )
    chunks = text_splitter.split_text(document)
````

### 2.3. 文档结构分割（Document Structure-based）
根据文档本身的格式（code、HTML、Markdown、JSON 等）进行分割。这类文档本身就具有明显的结构。       
详细代码参看 [`官方文档示例`](https://docs.langchain.com/oss/python/integrations/splitters#document-structure-based)

## 3 Embedding Model
各家Embedding 模型 集成参考：[`Top integrations`](https://docs.langchain.com/oss/python/integrations/text_embedding#top-integrations) [`All embedding models`](https://docs.langchain.com/oss/python/integrations/text_embedding#all-embedding-models)

## 4 Vector stores

已集成的 向量数据库：[`All vector stores`](https://docs.langchain.com/oss/python/integrations/vectorstores#all-vector-stores)

A vector store `stores embedded data` and performs `similarity search`.     

```mermaid  theme={null}
flowchart LR

    subgraph "📥 Indexing phase (store)"
        A[📄 Documents] --> B[🔢 Embedding model]
        B --> C[🔘 Embedding vectors]
        C --> D[(Vector store)]
    end

    subgraph "📤 Query phase (retrieval)"
        E[❓ Query text] --> F[🔢 Embedding model]
        F --> G[🔘 Query vector]
        G --> H[🔍 Similarity search]
        H --> D
        D --> I[📄 Top-k results]
    end
```

### Interface
LC 提供了一些统一的接口用于：
- add_documents - Add documents to the store.
- delete - Remove stored documents by ID.
- similarity_search - Query for semantically similar documents.

### Initialization
需要提供一个 embedding 模型：
````py
    from langchain_core.vectorstores import InMemoryVectorStore
    vector_store = InMemoryVectorStore(embedding=SomeEmbeddingModel())
````

### 添加、删除文档、相似性搜索

````py
    # 添加：文档对象及 id
    vector_store.add_documents(documents=[doc1, doc2], ids=["id1", "id2"])
    # 删除：id
    vector_store.delete(ids=["id1"])
    # 相似性搜索：query
    similar_docs = vector_store.similarity_search("your query here", k, filter={"source": "tweets"})
````

### Similarity metrics & indexing
相似度计算的 三类常见方法：
- Cosine similarity
- Euclidean distance
- Dot product       

或者 Vector stores 自身提供的一些方法。

### Metadata filtering
按元数据（例如来源、日期）筛选可以优化搜索结果：
````py
    vector_store.similarity_search(
    "query",
    k=3,
    filter={"source": "tweets"}
    )
````

## 5 Retrievers
检索器是一个接口，它根据非结构化查询返回文档。以下是一些常见的 retrievers ，全部的 retrievers 参考 [`All retrievers`](https://docs.langchain.com/oss/python/integrations/retrievers#all-retrievers)：
| Retriever                                                                                | Self-host | Cloud offering | Package                                                                                                                                                                               |
| ---------------------------------------------------------------------------------------- | --------- | -------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| [`AmazonKnowledgeBasesRetriever`](/oss/python/integrations/retrievers/bedrock)           | ❌         | ✅              | [`langchain-aws`](https://python.langchain.com/api_reference/aws/retrievers/langchain_aws.retrievers.bedrock.AmazonKnowledgeBasesRetriever.html)                                      |
| [`AzureAISearchRetriever`](/oss/python/integrations/retrievers/azure_ai_search)          | ❌         | ✅              | [`langchain-community`](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.azure_ai_search.AzureAISearchRetriever.html)                   |
| [`ElasticsearchRetriever`](/oss/python/integrations/retrievers/elasticsearch_retriever)  | ✅         | ✅              | [`langchain-elasticsearch`](https://python.langchain.com/api_reference/elasticsearch/retrievers/langchain_elasticsearch.retrievers.ElasticsearchRetriever.html)                       |
| [`VertexAISearchRetriever`](/oss/python/integrations/retrievers/google_vertex_ai_search) | ❌         | ✅              | [`langchain-google-community`](https://python.langchain.com/api_reference/google_community/vertex_ai_search/langchain_google_community.vertex_ai_search.VertexAISearchRetriever.html) |


### 5.1 milvus
以 milvus 为例。

